In [9]:
import os
import xarray as xr
import pandas as pd
import numpy as np

In [20]:
# parameter directory
param_dir = '/glade/work/afoster/FATES_calibration/parameter_files'

# default parameter file
fates_param_name = "fates_param_default.nc"

# csv with updated values for each pft and parameter
updated_values = '/glade/u/home/afoster/FATES_Calibration/final_parameters.csv'

In [35]:
# pfts were updated since I did this, plus no crops so dictionary to map between

FATES_INDEX = {"broadleaf_evergreen_tropical_tree": 1,
                "needleleaf_evergreen_extratrop_tree": 2,
                "needleleaf_colddecid_extratrop_tree": 3,
                "broadleaf_evergreen_extratrop_tree": 4,
                "broadleaf_hydrodecid_tropical_tree": 5,
                "broadleaf_colddecid_extratrop_tree": 6,
                "broadleaf_evergreen_extratrop_shrub": 7,
                "broadleaf_hydrodecid_extratrop_shrub": 8,
                "broadleaf_colddecid_extratrop_shrub": 9,
                "arctic_c3_grass": 10,
                "cool_c3_grass": 11,
                "c4_grass": 12,
                "c3_crop": 13,
                "c3_irrigated": 14}

FATES_INDEX_new = {"broadleaf_evergreen_tropical_tree": 1,
                "needleleaf_evergreen_extratrop_tree": 2,
                "needleleaf_colddecid_extratrop_tree": 3,
                "broadleaf_evergreen_extratrop_tree": 4,
                "broadleaf_hydrodecid_tropical_tree": 5,
                "broadleaf_colddecid_extratrop_tree": 6,
                "broadleaf_evergreen_extratrop_shrub": 7,
                "broadleaf_hydrodecid_extratrop_shrub": 8,
                "broadleaf_colddecid_extratrop_shrub": 9,
                "broadleaf_evergreen_arctic_shrub": 10,
                "broadleaf_colddecid_arctic_shrub": 11,
                "arctic_c3_grass": 12,
                "cool_c3_grass": 13,
                "c4_grass": 14}

# skip crops
skip_pfts = ['c3_crop', 'c3_irrigated']

In [72]:
# open default parameter
default_param_data = xr.open_dataset(os.path.join(param_dir, fates_param_name))

In [88]:
# copy new parameter file
new_param = default_param_data.copy(deep=False)

In [89]:
# update global variables first
new_param.fates_leaf_stomatal_model.values = 2  # mendlyn
new_param.fates_rad_model.values = 2  # two-stream
new_param.fates_maintresp_leaf_model.values = 2  # atkin

In [90]:
# now update pft-specific values from csv file
new_values = pd.read_csv(updated_values, index_col=[0])
params = sorted(np.unique(new_values.parameter.values))

In [91]:
for param in params:
    sub = new_values[new_values.parameter == param]

    default_val = new_param[param].values
    new_val = default_val
    pfts = sorted([pft for pft in np.unique(sub.pft) if pft not in skip_pfts])
    for pft in pfts:
        sub_val = sub[sub.pft == pft].value.values[0]
        pft_ind = FATES_INDEX_new[pft]-1
        if param == 'fates_leaf_vcmax25top':
            new_val[0, pft_ind] = round(sub_val, 3)
        else:
            new_val[pft_ind] = round(sub_val, 3)
    new_param[param].values = new_val

In [95]:
new_param.to_netcdf(os.path.join(param_dir, 'updated_param_file.nc'))